# Hugging Face Hub の Phi-3-mini モデル の LoRAによる効率的なファインチューニング手順
- Pythonコード生成に特化させる。
- https://github.com/microsoft/PhiCookBook/blob/main/code/04.Finetuning/Phi-3-finetune-lora-python.ipynb

## インストールとインポート

### ライブラリのインストールと読み込み
Installing and loading the libraries

In [ ]:
# This command is used to install and upgrade several Python packages using pip, Python's package installer.
# The '!' at the beginning allows us to run shell commands in the notebook.
# The '-qqq' option is used to make the installation process less verbose.
# '--upgrade' is used to ensure that the packages are upgraded to their latest versions if they are already installed.
# The packages being installed are:
# 'bitsandbytes' for efficient gradient accumulation,
# 'transformers' for using transformer models like Phi-3,
# 'peft' for efficient fine-tuning,
# 'accelerate' for easy distributed training,
# 'datasets' for loading and preprocessing datasets,
# 'trl' for reinforcement learning,
# 'flash_attn' for attention-based models.
# 'wandb' stands for Weights & Biases. It is a tool for machine learning experiment tracking, dataset versioning, and model management. It allows you to log and visualize metrics from your code, share findings, and reproduce experiments.
# 'torch' is a package that provides an open-source machine learning library used for building deep learning models.

# このコマンドは、pipを使用して複数の Python パッケージをインストールおよびアップグレードする
# ・先頭の '!' は、ノートブック上でシェルコマンドを実行するための記号
#   ・'-qqq' オプションは、インストール処理の出力を抑えて静かに実行させるためのものです。
#   ・'--upgrade' は、すでにインストールされているパッケージがある場合に、それを最新バージョンにアップグレードするために使用されます。
# ・インストール対象のパッケージは以下の通りです：
#   ・'bitsandbytes'：効率的な勾配蓄積のためのパッケージ
#   ・'transformers'：Phi-3 のような Transformer モデルを使用するためのライブラリ
#   ・'peft'：効率的なファインチューニングのためのライブラリ
#   ・'accelerate'：分散学習を簡単に行うためのライブラリ
#   ・'datasets'：データセットの読み込みと前処理のためのライブラリ
#   ・'trl'：強化学習（Reinforcement Learning）用のライブラリ
#   ・'flash_attn'：Attention ベースのモデルに関する処理のためのライブラリ
#   ・'wandb'：Weights & Biases の略で、機械学習の実験追跡、データセットのバージョン管理、モデル管理などを行うツール。コードからメトリクスを記録・可視化し、結果の共有や実験の再現を可能にします。
#   ・'torch'：深層学習モデルを構築するためのオープンソースの機械学習ライブラリ
# !pip install -qqq --upgrade bitsandbytes transformers peft accelerate datasets trl flash_attn torch wandb

In [ ]:
# そんな簡単にインストールできないのでバラして実行
!pip install --upgrade bitsandbytes
!pip install --upgrade transformers
!pip install --upgrade peft
!pip install --upgrade accelerate
!pip install --upgrade datasets
!pip install --upgrade trl

In [ ]:
# CUDAインストール環境にインストール
# pipとsetuptoolsをアップグレード
!pip install --upgrade pip setuptools

# --no-build-isolationは、この事前ビルド済みのものをダウンロードしてきてインストール
# 環境内のPython/CUDA/PyTorchなどのバージョンをチェックして選択している模様
# !pip install flash_attn --no-build-isolation

### 様々な問題対応
ビルドを並列実行する方法と、ビルド済みのものを持ってくる方法があるらしい。

#### ビルドを並列実行する方法
- [bug] build is verrrrrrrrrrrrrrrrrrrry slow · Issue #945 · Dao-AILab/flash-attention  
https://github.com/Dao-AILab/flash-attention/issues/945
```bash
$ top -d 10
top - 06:28:47 up  3:37,  2 users,  load average: 1.99, 2.01, 2.01
Tasks: 306 total,   3 running, 303 sleeping,   0 stopped,   0 zombie
%Cpu(s): 25.6 us,  0.3 sy,  0.0 ni, 74.1 id,  0.0 wa,  0.0 hi,  0.0 si,  0.0 st 
MiB Mem :  56191.6 total,  43355.9 free,   8707.5 used,   4902.0 buff/cache     
MiB Swap:      0.0 total,      0.0 free,      0.0 used.  47484.1 avail Mem 

    PID USER      PR  NI    VIRT    RES    SHR S  %CPU  %MEM     TIME+ COMMAND                               
   7944 seigi     20   0 7618672   1.8g  38648 R 100.0   3.3   2:52.44 cicc                                  
   7945 seigi     20   0 7618672   1.8g  38656 R 100.0   3.3   2:52.42 cicc                                  
   2657 xrdp      20   0   40156  28668   8856 S   2.6   0.0   1:31.88 xrdp                                  
   1854 seigi     20   0  485348 184864  87076 S   1.0   0.3   0:38.83 Xorg  
```
- CUDAプログラムのビルド中。ciccプロセスがCPUをフルに使っているが、全体としてCPUにもメモリにもまだ余裕がある。
- ロードアベレージは高め（2.0以上）、コア数が2以上あるなら並列数を上げると良い可能性がある。
- `export MAX_JOBS=N`で、ninjaビルドシステムが使用できる CPU コアを指定することができるらしい。
- `Standard_NC8as_T4_v3`のvCPU数は8らしい。`lscpu`で`CPU(s):8`、`On-line CPU(s) list:0-7`
```bash
!pip install packaging
!pip install ninja
!MAX_JOBS=8 pip install flash-attn --no-build-isolation
```
#### ビルド済みのものを持ってくる方法
- ColaboratoryでFlashAttentionをインストールするとめちゃめちゃ時間がかかっていた（過去）  
https://zenn.dev/kun432/scraps/98b51455d20927
  - https://x.com/mjun0812/status/1850706663982137615
  - https://github.com/mjun0812/flash-attention-prebuild-wheels
  - https://github.com/kun432/flash-attention-prebuild-wheels
```bash
pip install https://github.com/mjun0812/flash-attention-prebuild-wheels/releases/download/v0.1.0/flash_attn-2.7.4+cu128torch2.7-cp312-cp312-linux_x86_64.whl
```

In [ ]:
# 
!pip install packaging
!pip install ninja
# 
!pip cache purge
!mkdir -p $mnt/tmp

In [ ]:
# 「error: [Errno 28] No space left on device」の様なエラーになったので「TMPDIR」を追加
!TMPDIR=$mnt/tmp MAX_JOBS=4 pip install flash-attn --no-build-isolation

In [ ]:
!pip install --upgrade torch
!pip install --upgrade wandb

In [ ]:
# These commands are used to install two Python packages using pip, Python's package installer.
# The '!' at the beginning allows us to run shell commands in the notebook.
# 'huggingface_hub' is a library developed by Hugging Face that allows you to interact with the Hugging Face Model Hub.
# It provides functionalities to download and upload models, as well as other utilities.

# pip を使って 2 つのパッケージをインストールする
# 'huggingface_hub' は Hugging Face Model Hub からの モデルのダウンロードやアップロード、その他のユーティリティ機能を提供
!pip install huggingface_hub

# 'python-dotenv' is a library that allows you to specify environment variables in a .env file.
# It's useful for managing secrets and configuration settings for your application.

# 'python-dotenv' は、.env ファイルに環境変数を指定できるようにするライブラリでアプリケーションの機密情報や設定を管理するのに便利
!pip install python-dotenv

In [ ]:
# This command is used to install three Python packages using pip, Python's package installer.
# The '!' at the beginning allows us to run shell commands in the notebook.
# 'absl-py' is a library developed by Google that provides several utilities for Python development, such as logging and command line argument parsing.
# 'nltk' stands for Natural Language Toolkit. It is a leading platform for building Python programs to work with human language data. It provides easy-to-use interfaces to over 50 corpora and lexical resources.
# 'rouge_score' is a library for calculating the ROUGE (Recall-Oriented Understudy for Gisting Evaluation) score, which is commonly used for evaluating automatic summarization and machine translation systems.

# このコマンドは、pipdで3つのPythonパッケージをインストールする。
# ・'absl-py' はGoogleが開発したライブラリで、ロギングやコマンドライン引数のパースなど、Python開発向けのさまざまなユーティリティを提供
# ・'nltk' は Natural Language Toolkit の略で、人間の言語データを扱うPythonプログラムを構築するための代表的なプラットフォーム。50以上のコーパスや語彙資源に簡単にアクセスできるIFを提供。
# ・'rouge_score' は ROUGE（Recall-Oriented Understudy for Gisting Evaluation）スコアを計算するためのライブラリで、自動要約や機械翻訳システムの評価によく用いられる。
!pip install absl-py nltk rouge_score

In [9]:
# This command is used to list all installed Python packages and filter for the 'transformers' package.
# The '!' at the beginning allows us to run shell commands in the notebook.
# 'pip list' lists all installed Python packages.
# The '|' character is a pipe. It takes the output from the command on its left (in this case, 'pip list') and passes it as input to the command on its right.
# 'grep' is a command-line utility for searching plain-text data for lines that match a regular expression. Here it's used to filter the output of 'pip list' for lines that contain 'transformers.'.

# このコマンドは、インストールされているすべての Python パッケージを一覧表示し、その中から 'transformers' パッケージをフィルタリングする
# 先頭の '!' は、ノートブック内でシェルコマンドを実行する
# 'pip list' は、インストールされているすべての Python パッケージを一覧表示
# '|'（パイプ）は、左側のコマンド（この場合は 'pip list'）の出力を右側のコマンドに入力として渡す。
# 'grep' は、正規表現に一致する行をプレーンテキストデータから検索するコマンドラインユーティリティ。
# ここでは、'pip list' の出力から 'transformers' を含む行だけを抽出するために使われています。

# So, this command will list details of the 'transformers' package if it's installed.
# つまり、このコマンドを実行すると、'transformers' パッケージがインストールされている場合、その詳細が表示される。
!pip list | grep transformers.

transformers              4.52.3


In [10]:
# インストールされたパッケージを確認する。
!pip list

Package                   Version
------------------------- --------------
absl-py                   2.2.2
accelerate                1.7.0
aiohappyeyeballs          2.6.1
aiohttp                   3.11.18
aiosignal                 1.3.2
annotated-types           0.7.0
anyio                     4.9.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.5
attrs                     25.3.0
babel                     2.17.0
beautifulsoup4            4.13.4
bitsandbytes              0.45.5
bleach                    6.2.0
certifi                   2025.4.26
cffi                      1.17.1
charset-normalizer        3.4.2
click                     8.2.1
comm                      0.2.2
datasets                  3.6.0
debugpy                   1.8.14
decorator                 5.2.1
defusedxml                0.7.1
dill                      0.3.8
docker-pycreds            0.4.0
einops         

### ライブラリのインポート
Importing the libraries

In [ ]:
# This code block is importing necessary modules and functions for fine-tuning a language model.
# このコードブロックは、言語モデルのファインチューニングに必要なモジュールや関数をインポート

# 'randrange' is a function from the 'random' module that generates a random number within the specified range.
# 'randrange' は 'random' モジュールの関数で、指定された範囲内の乱数を生成
from random import randrange

# 'torch' is the PyTorch library, a popular open-source machine learning library for Python.
# 'torch' は PyTorch ライブラリで、Python 向けの人気のあるオープンソースの機械学習ライブラリ
import torch

# 'load_dataset' is a function from the 'datasets' library by Hugging Face which allows you to load a dataset.
# 'load_dataset' は、Hugging Face の 'datasets' ライブラリにある関数で、データセットを読み込むために使用
from datasets import load_dataset

# 'LoraConfig' and 'prepare_model_for_kbit_training' are from the 'peft' library. 
# 'LoraConfig' is used to configure the LoRA (Learning from Random Architecture) model.
# 'prepare_model_for_kbit_training' is a function that prepares a model for k-bit training.
# 'TaskType' contains differenct types of tasks supported by PEFT
# 'PeftModel' base model class for specifying the base Transformer model and configuration to apply a PEFT method to.

# 「peft」ライブラリから提供
# ・「LoraConfig」はLoRAモデルの構成を設定するために使われる。
# ・「prepare_model_for_kbit_training」は、kビット訓練に対応するようモデルを準備する関数。
# ・「TaskType」にはPEFTがサポートするさまざまなタスクの種類が含まれる。
# ・「PeftModel」は、ベースとなるTransformerモデルと構成を指定してPEFT手法を適用するための基底モデルクラス。
from peft import LoraConfig, prepare_model_for_kbit_training, TaskType, PeftModel

# Several classes and functions are imported from the 'transformers' library by Hugging Face.
# 'AutoModelForCausalLM' is a class that provides a generic transformer model for causal language modeling.
# 'AutoTokenizer' is a class that provides a generic tokenizer class.
# 'BitsAndBytesConfig' is a class for configuring the Bits and Bytes optimizer.
# 'TrainingArguments' is a class that defines the arguments used for training a model.
# 'set_seed' is a function that sets the seed for generating random numbers.
# 'pipeline' is a function that creates a pipeline that can process data and make predictions.

# Hugging Faceの「transformers」ライブラリから提供
# ・'AutoModelForCausalLM'は因果言語モデリングのための汎用的なTransformerモデルを提供
# ・'AutoTokenizer'は汎用的なTokenizerクラスを提供するクラス
# ・'BitsAndBytesConfig'はBits and Bytes最適化器の設定を行うためのクラス
# ・'TrainingArguments'はモデルの訓練に使用される引数を定義するクラス
# ・'set_seed'はランダム数生成のシードを設定する関数
# ・'pipeline'はデータを処理して予測を行うパイプラインを作成する関数
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    set_seed,
    pipeline
)

# 'SFTTrainer' is a class from the 'trl' library that provides a trainer for soft fine-tuning.
# 'SFTTrainer' は、ソフトファインチューニング用のトレーナーを提供する 'trl' ライブラリのクラス
from trl import SFTTrainer

## グローバルパラメータの設定
Setting Global Parameters

In [ ]:
# This code block is setting up the configuration for fine-tuning a language model.
# このコードブロックは、言語モデルのファインチューニングのための設定を行う。

# 'model_id' and 'model_name' are the identifiers for the pre-trained model that you want to fine-tune. 
# In this case, it's the 'Phi-3-mini-4k-instruct' model from Microsoft.
# 'model_id' および 'model_name' は、ファインチューニングしたい事前学習済みモデルの識別子
# この場合は、Microsoft の 'Phi-3-mini-4k-instruct' モデル

# Model Names 
# microsoft/Phi-3-mini-4k-instruct
# microsoft/Phi-3-mini-128k-instruct
# microsoft/Phi-3-small-8k-instruct
# microsoft/Phi-3-small-128k-instruct
# microsoft/Phi-3-medium-4k-instruct
# microsoft/Phi-3-medium-128k-instruct
# microsoft/Phi-3-vision-128k-instruct
# microsoft/Phi-3-mini-4k-instruct-onnx
# microsoft/Phi-3-mini-4k-instruct-onnx-web
# microsoft/Phi-3-mini-128k-instruct-onnx
# microsoft/Phi-3-small-8k-instruct-onnx-cuda
# microsoft/Phi-3-small-128k-instruct-onnx-cuda
# microsoft/Phi-3-medium-4k-instruct-onnx-cpu
# microsoft/Phi-3-medium-4k-instruct-onnx-cuda
# microsoft/Phi-3-medium-4k-instruct-onnx-directml
# microsoft/Phi-3-medium-128k-instruct-onnx-cpu
# microsoft/Phi-3-medium-128k-instruct-onnx-cuda
# microsoft/Phi-3-medium-128k-instruct-onnx-directml
# microsoft/Phi-3-mini-4k-instruct-gguf
model_id = "microsoft/Phi-3-mini-4k-instruct"
model_name = "microsoft/Phi-3-mini-4k-instruct"

# 'dataset_name' is the identifier for the dataset that you want to use for fine-tuning. 
# In this case, it's the 'python_code_instructions_18k_alpaca' dataset from iamtarun (Ex: iamtarun/python_code_instructions_18k_alpaca).
# Update Dataset Name to your dataset name

# 'dataset_name' は、ファインチューニングに使用したいデータセットの識別子
# "iamtarun/python_code_instructions_18k_alpaca"には、Python 言語で記述された問題の説明とコードが含まれる。
# https://huggingface.co/datasets/iamtarun/python_code_instructions_18k_alpaca
dataset_name = "iamtarun/python_code_instructions_18k_alpaca" # "Insert your dataset name here"

# 'dataset_split' is the split of the dataset that you want to use for training. 
# In this case, it's the 'train' split.
# 'dataset_split' は、学習に使用したいデータセットの分割を指定
# この場合は 'train'（訓練）分割（を使用するように指定）
dataset_split = "train"

# 'new_model' is the name that you want to give to the fine-tuned model.
# 'new_model' はファインチューニングしたモデルに付けたい名前
new_model = "Phi-3-mini-4k-finetuned"

# 'hf_model_repo' is the repository on the Hugging Face Model Hub where the fine-tuned model will be saved. Update UserName to your Hugging Face Username
# 'hf_model_repo' はファインチューニングされたモデルを保存するための Hugging Face Model Hub 上のリポジトリ。UserName を自身の Hugging Face ユーザー名に更新
hf_userName = "nishi74322014"
hf_model_repo = hf_userName + "/" + new_model

# 'device_map' is a dictionary that maps the model to the GPU device. 
# In this case, the entire model is loaded on GPU 0.
# 'device_map' は、モデルをGPUデバイスに割り当てる辞書
# この場合、モデル全体がGPU 0に読み込まれます。
device_map = {"": 0}

# The following are parameters for the LoRA (Learning from Random Architecture) model.
# 以下はLoRA（Learning from Random Architecture）モデルのパラメタ

# 'lora_r' is the dimension of the LoRA attention.
# 'lora_r' は LoRAアテンションの次元数
lora_r = 16

# 'lora_alpha' is the alpha parameter for LoRA scaling.
# 'lora_alpha' は LoRA のスケーリングに用いられる α（アルファ）パラメタ
lora_alpha = 16

# 'lora_dropout' is the dropout probability for LoRA layers.
# 'lora_dropout' は LoRA 層に対するドロップアウトの確率
lora_dropout = 0.05

# 'target_modules' is a list of the modules in the model that will be replaced with LoRA layers.
# 'target_modules' は、LoRA層に置き換えられるモデル内のモジュールのリスト
target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]

# 'set_seed' is a function that sets the seed for generating random numbers, which is used for reproducibility of the results.
# 'set_seed' はランダムな数を生成するためのシードを設定する関数で結果の再現性を確保するために使用される。
set_seed(1234)

## Huggingface Hubに接続する
Connect to Huggingface Hub

IMPORTANT: The upcoming section's execution will vary based on your code execution environment and the configuration of your API Keys.  
重要: 次のセクションの実行は、コード実行環境と API キーの構成によって異なる。

### Hugging Face Hubへのインタラクティブログインが可能
Interactive login to Hugging Face Hub is possible.

In [ ]:
# This code block is used to log in to the Hugging Face Model Hub from a notebook.
# このコードブロックは、notebookからHugging Face Model Hubにログインする。

# 'notebook_login' is a function from the 'huggingface_hub' library that opens a new browser window 
# where you can log in to your Hugging Face account. After logging in, 
# your Hugging Face token will be stored in a configuration file on your machine, 
# which allows you to interact with the Hugging Face Model Hub from your notebook.

# 'notebook_login'は、'huggingface_hub'ライブラリの関数で、新しいブラウザウィンドウを開き、 
# そこでHugging Faceアカウントにログインできます。ログイン後、 
# あなたのHugging Faceトークンは、あなたのマシンの設定ファイルに保存され、 
# ノートブックからHugging Face Model Hubとやり取りできるようになる。
from huggingface_hub import notebook_login

# Call the 'notebook_login' function to start the login process.
# 'notebook_login' 関数を呼び出してログインプロセスを開始。
notebook_login()

### または、Hugging Face トークンを含む .env ファイルを提供できる。
Alternatively, you can supply a .env file that contains the Hugging Face token.

In [ ]:
# This code block is used to log in to the Hugging Face Model Hub using an API token stored in an environment variable.
# このコードブロックは、環境変数に格納されたAPIトークンを使用してHugging Face Model Hubにログインする。

# 'login' is a function from the 'huggingface_hub' library that logs you in to the Hugging Face Model Hub using an API token.
# 'login'は'huggingface_hub'ライブラリの関数で、APIトークンを使用してHugging Face Model Hubにログインする。
from huggingface_hub import login

# 'load_dotenv' is a function from the 'python-dotenv' library that loads environment variables from a .env file.
# 'load_dotenv' は 'python-dotenv' ライブラリの関数で、.env ファイルから環境変数をロード
from dotenv import load_dotenv

# 'os' is a standard Python library that provides functions for interacting with the operating system.
# 'os' は、オペレーティングシステムと対話するための標準の Python ライブラリ
import os

# Call the 'load_dotenv' function to load the environment variables from the .env file.
# 'load_dotenv' 関数を呼び出して、.env ファイルから環境変数をロード
load_dotenv()

# Call the 'login' function with the 'HF_HUB_TOKEN' environment variable to log in to the Hugging Face Model Hub.
# 'os.getenv' is a function that gets the value of an environment variable.
# 'HF_HUB_TOKEN' 環境変数を使用して 'login' 関数を呼び出し、Hugging Face Model Hub にログイン
# 'os.getenv' は環境変数の値を取得する関数
login(token=os.getenv("HF_HUB_TOKEN"))

## データの準備

### 命令セット付きでデータセットを読み込む
Load the dataset with the instruction set

In [ ]:
# This code block is used to load a dataset from the Hugging Face Dataset Hub, print its size, and show a random example from the dataset.
# このコードブロックは、Hugging Face Dataset Hub からデータセットを読み込み、そのサイズ、ランダムなサンプルを表示

# 'load_dataset' is a function from the 'datasets' library that loads a dataset from the Hugging Face Dataset Hub.
# 'dataset_name' is the name of the dataset to load, and 'dataset_split' is the split of the dataset to load (e.g., 'train', 'test').
# 'load_dataset' は 'datasets' ライブラリの関数で、Hugging Face Dataset Hub からデータセットを読み込む。
# 'dataset_name' は読み込むデータセットの名前で、'dataset_split' は読み込むデータセット分割（例：'train', 'test'）を表す。
dataset = load_dataset(dataset_name, split=dataset_split)

# The 'len' function is used to get the size of the dataset, which is then printed.
# 'len' 関数を使ってデータセットのサイズを取得し、それを表示。
print(f"dataset size: {len(dataset)}")

# 'randrange' is a function from the 'random' module that generates a random number within the specified range.
# Here it's used to select a random example from the dataset, which is then printed.
# 'randrange' は 'random' モジュールの関数で、指定した範囲内のランダムな数値を生成
# ここではデータセットからランダムなサンプルを選び、それを表示
print(dataset[randrange(len(dataset))])

In [ ]:
# This line of code is used to display the structure of the 'dataset' object.
# By simply writing the name of the object, Python will call its 'repr' (representation) method, 
# which returns a string that describes the object. 
# For a Hugging Face 'Dataset' object, this will typically show information such as the number of rows, 
# the column names, and the types of the data in each column.

# このコード行は 'dataset' オブジェクトの構造を表示する
# オブジェクトで、'repr'（表現）メソッドを呼び出し、オブジェクトを説明する文字列を返す。
# Hugging Face の 'Dataset' オブジェクトの場合、通常は行数、列名、型の情報が表示される。
dataset

### トークナイザーを読み込んでデータセットを準備
Load the tokenizer to prepare the dataset

In [ ]:
# This code block is used to load a tokenizer from the Hugging Face Model Hub.
# このコードブロックは、Hugging Face Model Hub からトークナイザーを読み込む

# 'tokenizer_id' is set to the 'model_id', which is the identifier for the pre-trained model.
# This assumes that the tokenizer associated with the model has the same identifier as the model.
# 'tokenizer_id' は 'model_id' に設定されています。これは事前学習済みモデルの識別子です。
# モデルに関連付けられたトークナイザーがモデルと同じ識別子を持っていると仮定しています。
tokenizer_id = model_id

# 'AutoTokenizer.from_pretrained' is a method that loads a tokenizer from the Hugging Face Model Hub.
# 'tokenizer_id' is passed as an argument to specify which tokenizer to load.
# 'AutoTokenizer.from_pretrained' は、Hugging Face Model Hub からトークナイザーを読み込むメソッドで
# どのトークナイザーを読み込むかを指定するために 'tokenizer_id' が引数として渡される。
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)

# 'tokenizer.padding_side' is a property that specifies which side to pad when the input sequence is shorter than the maximum sequence length.
# Setting it to 'right' means that padding tokens will be added to the right (end) of the sequence.
# This is done to prevent warnings that can occur when the padding side is not explicitly set.
# 'tokenizer.padding_side' は、入力シーケンスが最大長より短い場合に、どちら側にパディングを追加するかを指定するプロパティ。
# 'right' に設定すると、パディングトークンがシーケンスの右（末尾）に追加
# パディングの方向を明示的に設定しないと警告が出ることがあるため、この設定を行う。
tokenizer.padding_side = 'right'

### データセットをChatML形式に適合させる。
Function to create the appropiate format for our model.

We are going to adapt our dataset to the ChatML format.  
モデルに適した形式を作成する関数。

In [ ]:
# This code block defines two functions that are used to format the dataset for training a chat model.
# このコードブロックでは、チャットモデルの学習用データセットを整形するための2つの関数を定義

# 'create_message_column' is a function that takes a row from the dataset and returns a dictionary 
# with a 'messages' key and a list of 'user' and 'assistant' messages as its value.
# 'create_message_column' は、データセットの各行を受け取り 'messages' というキーを持つ辞書を返す。
# この 'messages' の値は、'user' と 'assistant' のメッセージをリスト形式で格納したもの。
def create_message_column(row):
    # Initialize an empty list to store the messages.
    # メッセージを格納するための空のリストを初期化する。
    messages = []
    
    # Create a 'user' message dictionary with 'content' and 'role' keys.
    # 「content」と「role」のキーを持つ「user」メッセージの辞書を作成
    user = {
        "content": f"{row['instruction']}\n Input: {row['input']}",
        "role": "user"
    }
    
    # Append the 'user' message to the 'messages' list.
    # 'user' メッセージを 'messages' リストに追加
    messages.append(user)
    
    # Create an 'assistant' message dictionary with 'content' and 'role' keys.
    # 「content」と「role」キーを持つ「assistant」メッセージ辞書を作成
    assistant = {
        "content": f"{row['output']}",
        "role": "assistant"
    }
    
    # Append the 'assistant' message to the 'messages' list.
    # 'assistant'メッセージを'messages'リストに追加
    messages.append(assistant)
    
    # Return a dictionary with a 'messages' key and the 'messages' list as its value.
    # 'messages'というキーとその値として'messages'リストを持つ辞書を返す。
    return {"messages": messages}

# 'format_dataset_chatml' is a function that takes a row from the dataset and returns a dictionary 
# with a 'text' key and a string of formatted chat messages as its value.
# 'format_dataset_chatml' はデータセットから行を取り、'text' キーとその値としてフォーマットされたチャットメッセージの文字列を持つ辞書を返す関数。
def format_dataset_chatml(row):
    # 'tokenizer.apply_chat_template' is a method that formats a list of chat messages into a single string.
    # 'add_generation_prompt' is set to False to not add a generation prompt at the end of the string.
    # 'tokenize' is set to False to return a string instead of a list of tokens.
    # 'tokenizer.apply_chat_template' は、チャットメッセージのリストを1つの文字列に整形するメソッド
    # 'add_generation_prompt' を False に設定すると、文字列の末尾に生成用のプロンプトを追加しない。
    # 'tokenize' を False に設定すると、トークンのリストではなく文字列を返す。    
    return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}


### ChatMLフォーマットをデータセットに適用
Apply the ChatML format to our dataset

The code block is used to prepare a dataset for training a chat model.  
このコードブロックは、チャットモデルのトレーニング用にデータセットを準備

The dataset.map(create_message_column) line applies the create_message_column function to each example in the dataset.  
This function takes a row from the dataset and transforms it into a dictionary with a 'messages' key. The value of this key is a list of 'user' and 'assistant' messages.  
`dataset.map(create_message_column)`の行は、`create_message_column`関数をデータセットの各例に適用。  
この関数は、データセットの行を受け取り、それを「messages」キーを持つ辞書に変換。このキーの値は「user」と「assistant」のメッセージのリスト。

The 'user' message is created by combining the 'instruction' and 'input' fields from the row, while the 'assistant' message is created from the 'output' field of the row.  
These messages are appended to the 'messages' list in the order of 'user' and 'assistant'.  
「user」メッセージは、行の「instruction」および「input」フィールドを組み合わせて作成され、「assistant」メッセージは行の「output」フィールドから作成される。  
これらのメッセージは、「user」と「assistant」の順で「messages」リストに追加される。

The dataset_chatml.map(format_dataset_chatml) line then applies the format_dataset_chatml function to each example in the updated dataset.  
This function takes a row from the dataset and transforms it into a dictionary with a 'text' key. The value of this key is a string of formatted chat messages.  
次に、`dataset_chatml.map(format_dataset_chatml)`の行が更新されたデータセットの各例に`format_dataset_chatml`関数を適用。
この関数は、データセットの行を受け取り、それを「text」キーを持つ辞書に変換。このキーの値はフォーマットされたチャットメッセージの文字列。

The tokenizer.apply_chat_template method is used to format the list of chat messages into a single string.  
The 'add_generation_prompt' parameter is set to False to avoid adding a generation prompt at the end of the string,  
and the 'tokenize' parameter is set to False to return a string instead of a list of tokens.  
`tokenizer.apply_chat_template`メソッドは、チャットメッセージのリストを単一の文字列にフォーマットする。  
「add_generation_prompt」パラメータは、文字列の最後に生成プロンプトを追加しないように`False`に設定され、  
「tokenize」パラメータはトークンのリストではなく文字列を返すように`False`に設定される。

The result of these operations is a dataset where each example is a dictionary with a 'text' key and a string of formatted chat messages as its value.  
This format is suitable for training a chat model.  
これらの操作の結果、各例が「text」キーとフォーマットされたチャットメッセージの文字列を値として持つ辞書であるデータセットが得られる。  
このフォーマットは、チャットモデルのトレーニングに適している。

In [ ]:
# This code block is used to prepare the 'dataset' for training a chat model.
# このコードブロックは、チャットモデルの学習用に「dataset」を準備する

# 'dataset.map' is a method that applies a function to each example in the 'dataset'.
# 'create_message_column' is a function that formats each example into a 'messages' format suitable for a chat model.
# The result is a new 'dataset_chatml' with the formatted examples.

# 「dataset.map」は「dataset」の各サンプルに関数を適用するメソッド
# 「create_message_column」は、各サンプルをチャットモデルに適した「messages」形式に整形する関数
# 結果として、整形されたサンプルを含む新しい「dataset_chatml」が得られる。

dataset_chatml = dataset.map(create_message_column)

# 'dataset_chatml.map' is a method that applies a function to each example in the 'dataset_chatml'.
# 'format_dataset_chatml' is a function that further formats each example into a single string of chat messages.
# The result is an updated 'dataset_chatml' with the further formatted examples.

# 「dataset_chatml.map」は、「dataset_chatml」内の各サンプルに関数を適用するメソッド
# 「format_dataset_chatml」は、各サンプルをチャットメッセージの単一の文字列にさらに整形する関数
# その結果、より整形されたサンプルで更新された「dataset_chatml」が得られる。

dataset_chatml = dataset_chatml.map(format_dataset_chatml)

In [ ]:
# This line of code is used to access and display the first example from the 'dataset_chatml'.
# このコード行は、'dataset_chatml'から最初の例を取得して表示する

# 'dataset_chatml[0]' uses indexing to access the first example in the 'dataset_chatml'.
# In Python, indexing starts at 0, so 'dataset_chatml[0]' refers to the first example.
# The result is a dictionary with a 'text' key and a string of formatted chat messages as its value.

# 'dataset_chatml[0]' はインデックス指定により、'dataset_chatml'の最初の例にアクセスする。
# Pythonではインデックスは0から始まるため、'dataset_chatml[0]'は最初の例を指す。
# 結果として得られるのは、'text'というキーを持ち、その値としてフォーマットされたチャットメッセージの文字列を含む辞書
dataset_chatml[0]

### データセットを訓練用とテスト用に分割する
Split the dataset into a train and test sets

In [ ]:
# This code block is used to split the 'dataset_chatml' into training and testing sets.
# このコードブロックは、'dataset_chatml' をトレーニング用とテスト用に分割する

# 'dataset_chatml.train_test_split' is a method that splits the 'dataset_chatml' into a training set and a testing set.
# 'test_size' is a parameter that specifies the proportion of the 'dataset_chatml' to include in the testing set. Here it's set to 0.05, meaning that 5% of the 'dataset_chatml' will be included in the testing set.
# 'seed' is a parameter that sets the seed for the random number generator. This is used to ensure that the split is reproducible. Here it's set to 1234.
# 'dataset_chatml.train_test_split' は、'dataset_chatml' をトレーニングセットとテストセットに分割するメソッド
# 'test_size' は、'dataset_chatml' のうちテストセットに含める割合を指定するパラメタ。ここでは 0.05 、つまり 'dataset_chatml' の 5% がテストセットに含まれることを意味する。
# 'seed' は乱数生成器のシードを設定するパラメタ。これは分割の再現性を確保するために使用され、ここでは 1234 に設定されている。
dataset_chatml = dataset_chatml.train_test_split(test_size=0.05, seed=1234)

# This line of code is used to display the structure of the 'dataset_chatml' after the split.
# It will typically show information such as the number of rows in the training set and the testing set.
# このコード行は、分割後の 'dataset_chatml' の構造を表示するためのものです。
# 通常は、トレーニングセットとテストセットの行数などの情報が表示されます。
dataset_chatml
     

## ファインチューニングの設定
Instruction fine-tune a Phi-3-mini model using LORA and trl  
Phi-3-miniモデルをLORAとtrlを用いてInstructionチューニングする

### まず、GPUを特定してみましょう。
First, we try to identify out GPU

In [ ]:
# This code block is used to set the compute data type and attention implementation based on whether bfloat16 is supported on the current CUDA device.
# このコードブロックは、現在のCUDAデバイスでbfloat16がサポートされているかどうかに基づいて、計算データ型とアテンションの実装方法を設定するためのもの。

# 'torch.cuda.is_bf16_supported()' is a function that checks if bfloat16 is supported on the current CUDA device.
# If bfloat16 is supported, 'compute_dtype' is set to 'torch.bfloat16' and 'attn_implementation' is set to 'flash_attention_2'.
# 'torch.cuda.is_bf16_supported()' は、現在のCUDAデバイスでbfloat16がサポートされているかを確認する関数。
# bfloat16がサポートされている場合、'compute_dtype' は 'torch.bfloat16' に、'attn_implementation' は 'flash_attention_2' に設定される。
if torch.cuda.is_bf16_supported():
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
# If bfloat16 is not supported, 'compute_dtype' is set to 'torch.float16' and 'attn_implementation' is set to 'sdpa'.
# bfloat16 がサポートされていない場合、'compute_dtype' は 'torch.float16' に設定され、'attn_implementation' は 'sdpa' に設定される。
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'

# This line of code is used to print the value of 'attn_implementation', which indicates the chosen attention implementation.
# このコード行は、「attn_implementation」の値を表示する。これは、選択されたアテンションの実装方式を示す。
print(attn_implementation)   

### トークナイザーとモデルをロード
Load the tokenizer and model to finetune

In [ ]:
# This code block is used to load a pre-trained model and its associated tokenizer from the Hugging Face Model Hub.
# このコードブロックは、Hugging Face Model Hubから事前学習済みのモデルと関連するトークナイザーを読み込む

# 'model_name' is set to the identifier of the pre-trained model.
# 'model_name'は、事前学習済みモデルの識別子に設定（設定済みのためコメントアウト）
# model_name = "microsoft/Phi-3-mini-4k-instruct"

# 'AutoTokenizer.from_pretrained' is a method that loads a tokenizer from the Hugging Face Model Hub.
# 'model_id' is passed as an argument to specify which tokenizer to load.
# 'trust_remote_code' is set to True to trust the remote code in the tokenizer files.
# 'add_eos_token' is set to True to add an end-of-sentence token to the tokenizer.
# 'use_fast' is set to True to use the fast version of the tokenizer.

# 'AutoTokenizer.from_pretrained' は、Hugging Face Model Hub からトークナイザーをロードするメソッド
# ・'model_id' は、ロードするトークナイザーを指定
# ・'trust_remote_code' は、トークナイザーファイル内のリモートコード信頼
# ・'add_eos_token' は、トークナイザーに文の終了トークン追加
# ・'use_fast' は、トークナイザーの高速バージョン使用
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, add_eos_token=True, use_fast=True)

# The padding token is set to the unknown token.
# パディングトークンは未知のトークンに設定。
tokenizer.pad_token = tokenizer.unk_token

# The ID of the padding token is set to the ID of the unknown token.
# パディングトークンのIDは、未知のトークンのIDに設定。
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)

# The padding side is set to 'left', meaning that padding tokens will be added to the left (start) of the sequence.
# パディングの位置は「左」に設定（パディングトークンはシーケンスの左側（開始部分）に追加される。
tokenizer.padding_side = 'left'

# 'AutoModelForCausalLM.from_pretrained' is a method that loads a pre-trained model for causal language modeling from the Hugging Face Model Hub.
# 'model_id' is passed as an argument to specify which model to load.
# 'torch_dtype' is set to the compute data type determined earlier.
# 'trust_remote_code' is set to True to trust the remote code in the model files.
# 'device_map' is passed as an argument to specify the device mapping for distributed training.
# 'attn_implementation' is set to the attention implementation determined earlier.

# 'AutoModelForCausalLM.from_pretrained' は、Hugging Face Model Hub から因果言語モデリング用の事前学習済みモデルを読み込むメソッド
# ・'model_id' は、読み込むモデルを指定する
# ・'torch_dtype' は、先に決定された計算データ型に設定
# ・'trust_remote_code' は、モデルファイル内のリモートコード信頼
# ・'device_map' は、分散トレーニングのためのデバイス割り当て
# ・'attn_implementation' は、先に決定された attention の実装方法
model = AutoModelForCausalLM.from_pretrained(
          model_id, torch_dtype=compute_dtype, trust_remote_code=True, device_map=device_map,
          attn_implementation=attn_implementation
)

### LoRAプロパティの設定
Configure the LoRA properties

The SFTTrainer offers seamless integration with peft, simplifying the process of instruction tuning LLMs.  
All we need to do is create our LoRAConfig and supply it to the trainer.  
However, before initiating the training process, we must specify the hyperparameters we intend to use, which are defined in TrainingArguments.

SFTTrainerはpeftとのシームレスな統合を提供し、LLMの指示調整プロセスを簡素化します。  
必要なのは、LoRAConfigを作成し、それをトレーナーに提供することだけです。  
しかし、トレーニングプロセスを開始する前に、使用するハイパーパラメータを指定する必要があり、これらはTrainingArgumentsで定義されています。

In [ ]:
# This code block is used to define the training arguments for the model.
# このコードブロックは、モデルのトレーニング用の引数を定義する

# 'TrainingArguments' is a class that holds the arguments for training a model.
# 'output_dir' is the directory where the model and its checkpoints will be saved.
# 'evaluation_strategy' is set to "steps", meaning that evaluation will be performed after a certain number of training steps.
# 'do_eval' is set to True, meaning that evaluation will be performed.
# 'optim' is set to "adamw_torch", meaning that the AdamW optimizer from PyTorch will be used.
# 'per_device_train_batch_size' and 'per_device_eval_batch_size' are set to 8, meaning that the batch size for training and evaluation will be 8 per device.
# 'gradient_accumulation_steps' is set to 4, meaning that gradients will be accumulated over 4 steps before performing a backward/update pass.
# 'log_level' is set to "debug", meaning that all log messages will be printed.
# 'save_strategy' is set to "epoch", meaning that the model will be saved after each epoch.
# 'logging_steps' is set to 100, meaning that log messages will be printed every 100 steps.
# 'learning_rate' is set to 1e-4, which is the learning rate for the optimizer.
# 'fp16' is set to the opposite of whether bfloat16 is supported on the current CUDA device.
# 'bf16' is set to whether bfloat16 is supported on the current CUDA device.
# 'eval_steps' is set to 100, meaning that evaluation will be performed every 100 steps.
# 'num_train_epochs' is set to 3, meaning that the model will be trained for 3 epochs.
# 'warmup_ratio' is set to 0.1, meaning that 10% of the total training steps will be used for the warmup phase.
# 'lr_scheduler_type' is set to "linear", meaning that a linear learning rate scheduler will be used.
# 'report_to' is set to "wandb", meaning that training and evaluation metrics will be reported to Weights & Biases.
# 'seed' is set to 42, which is the seed for the random number generator.
# このコードブロックは、モデルのトレーニング用の引数を定義するためのものです。

# 'TrainingArguments' は、モデルのトレーニングに関する引数を保持するクラス
# ・'output_dir' は、モデルおよびそのチェックポイントが保存されるディレクトリ
# ・'evaluation_strategy' = "steps" は一定のステップごとに評価を実施する
# ・'do_eval' = True は評価する実施する。
# ・'optim' = "adamw_torch" はPyTorch の AdamW オプティマイザを使用
# ・'per_device_train_batch_size' および 'per_device_eval_batch_size' = 8 はトレーニングおよび評価時のバッチサイズ = 8
# ・'gradient_accumulation_steps' = 4 は、勾配が4ステップ分蓄積されてから逆伝播および更新を実行
# ・'log_level' = "debug" は、すべてのログメッセージを表示
# ・'save_strategy' = "epoch" は、各エポック後にモデルを保存
# ・'logging_steps' = 100 は、100ステップごとにログメッセージを表示
# ・'learning_rate' = 1e-4 は、オプティマイザに使われる学習率
# ・'fp16' は現在の CUDA デバイスが bfloat16 をサポートしているかどうかに応じて逆の値に設定
# ・'bf16' は現在の CUDA デバイスが bfloat16 をサポートしているかどうかに基づいて設定
# ・'eval_steps' = 100 は、100ステップごとに評価を実行
# ・'num_train_epochs' = 3 は、モデルが3エポック分トレーニング
# ・'warmup_ratio' = 0.1 は、トレーニング全体の10%をウォームアップフェーズとする。
# ・'lr_scheduler_type' = "linear" は、線形スケジューラを使用
# ・'report_to' = "wandb" は、トレーニングおよび評価の指標 Weights & Biases を有効化
# ・'seed' = 42 は、乱数生成器のシード
args = TrainingArguments(
        output_dir="./phi-3-mini-LoRA",
        evaluation_strategy="steps",
        do_eval=True,
        optim="adamw_torch",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        per_device_eval_batch_size=8,
        log_level="debug",
        save_strategy="epoch",
        logging_steps=100,
        learning_rate=1e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        eval_steps=100,
        num_train_epochs=3,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
        report_to="wandb",
        seed=42,
)

# LoraConfig object is created with the following parameters:
# 'r' (rank of the low-rank approximation) is set to 16,
# 'lora_alpha' (scaling factor) is set to 16,
# 'lora_dropout' dropout probability for Lora layers is set to 0.05,
# 'task_type' (set to TaskType.CAUSAL_LM indicating the task type),
# 'target_modules' (the modules to which LoRA is applied) choosing linear layers except the output layer.

# LoraConfig オブジェクトを以下のパラメタで作成：
# ・'r'（低ランク近似のランク）は 16 に設定
# ・'lora_alpha'（スケーリング係数）は 16 に設定
# ・'lora_dropout'（LoRA層のドロップアウト確率）は 0.05 に設定
# ・'task_type'（タスクの種類）は TaskType.CAUSAL_LM に設定（これは因果言語モデルタスクを示す）
# ・'target_modules'（LoRA を適用するモジュール）には、出力層を除いた線形層を選択
peft_config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        task_type=TaskType.CAUSAL_LM,
        target_modules=target_modules,
)

### wandb と接続を確立
Establish Connection with wandb and Initiate the Project and Experiment
wandb と接続を確立し、プロジェクトと実験を開始する

In [ ]:
# This code block is used to initialize Weights & Biases (wandb), a tool for tracking and visualizing machine learning experiments.
# このコードブロックは、Weights & Biases（wandb）という機械学習実験のトラッキングと可視化ツールを初期化

# 'import wandb' is used to import the wandb library.
# 'import wandb' で wandb ライブラリをインポート
import wandb

# 'wandb.login()' is a method that logs you into your Weights & Biases account.
# If you're not already logged in, it will prompt you to log in.
# Once you're logged in, you can use Weights & Biases to track and visualize your experiments.

# 'wandb.login()' は、Weights & Biases のアカウントにログインするためのメソッド
# まだログインしていない場合は、ログインを促すプロンプトが表示される。
# ログインすると、Weights & Biases を使って実験のトラッキングと可視化ができる。
wandb.login()

In [ ]:
# This code block is used to initialize a Weights & Biases (wandb) run.
# このコードブロックは、Weights & Biases（wandb）の実行を初期化

# 'project_name' is set to the name of the project in Weights & Biases.
# 'project_name' は、Weights & Biases 上のプロジェクト名を設定
project_name = "Phi3-mini-ft-python-code"

# 'wandb.init' is a method that initializes a new Weights & Biases run.
# 'project' is set to 'project_name', meaning that the run will be associated with this project.
# 'name' is set to "phi-3-mini-ft-py-3e", which is the name of the run.
# Each run has a unique name which can be used to identify it in the Weights & Biases dashboard.

# 'wandb.init' は、新しい Weights & Biases の実行を初期化するメソッド
# ・'project' に 'project_name' を設定、実行がプロジェクト名に紐づけられる。
# ・'name' に "phi-3-mini-ft-py-3e" を設定、これはこの実行の名前。
# 各実行には一意の名前があり、Weights & Biases のダッシュボード上で識別するために使用。
wandb.init(project=project_name, name = "phi-3-mini-ft-py-3e")

### SFTTrainerを構築
We now possess all the necessary components to construct our SFTTrainer and commence the training of our model.  
私たちは現在、SFTTrainer を構築し、モデルのトレーニングを開始するために必要なすべての要素を揃えています。

In [ ]:
# This code block is used to initialize the SFTTrainer, which is used to train the model.
# このコードブロックは、モデルを学習させるために使用される SFTTrainer を初期化する

# 'model' is the model that will be trained.
# 'train_dataset' and 'eval_dataset' are the datasets that will be used for training and evaluation, respectively.
# 'peft_config' is the configuration for peft, which is used for instruction tuning.
# 'dataset_text_field' is set to "text", meaning that the 'text' field of the dataset will be used as the input for the model.
# 'max_seq_length' is set to 512, meaning that the maximum length of the sequences that will be fed to the model is 512 tokens.
# 'tokenizer' is the tokenizer that will be used to tokenize the input text.
# 'args' are the training arguments that were defined earlier.

# ・'model' は学習対象のモデルです。
# ・'train_dataset' と 'eval_dataset' は、それぞれ学習と評価に使用されるデータセット
# ・'peft_config' は instruction tuning に使用される peft の設定
# ・'dataset_text_field' は "text" に設定されており、データセットの 'text' フィールドがモデルへの入力として使用される。
# ・'max_seq_length' は 512 に設定されており、モデルに入力されるシーケンスの最大長が 512 トークンであることを意味する。
# ・'tokenizer' は入力テキストをトークナイズするために使用されるトークナイザー
# ・'args' は前のステップで定義された学習用引数
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset_chatml['train'],
        eval_dataset=dataset_chatml['test'],
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=args,
)

## ファインチューニング実行
と、モデルの保存も行う。

### Trainer インスタンスで train() メソッドを呼び出して、モデルのトレーニング プロセスを初期化
Initiate the model training process by invoking the train() method on our Trainer instance.

In [ ]:
# This code block is used to train the model and save it locally.
# このコードブロックは、モデルのトレーニングとローカルへの保存に使用

# 'trainer.train()' is a method that starts the training of the model.
# It uses the training dataset, evaluation dataset, and training arguments that were provided when the trainer was initialized.

# 'trainer.train()' は、モデルのトレーニングを開始するメソッド
# トレーナーの初期化時に指定されたトレーニングデータセット、評価データセット、およびトレーニング引数を使用
trainer.train()

# 'trainer.save_model()' is a method that saves the trained model locally.
# The model will be saved in the directory specified by 'output_dir' in the training arguments.

# 'trainer.save_model()' は、トレーニング済みモデルをローカルに保存するメソッド
# モデルは、トレーニング引数で指定された 'output_dir' ディレクトリに保存される
trainer.save_model()

### アダプターをHugging Face Hubに保管
Store the adapter on the Hugging Face Hub

In [ ]:
# This code block is used to save the adapter to the Hugging Face Model Hub.
# このコードブロックは、アダプターを Hugging Face Model Hub に保存するために使用されます。

# 'trainer.push_to_hub' is a method that pushes the trained model (or adapter in this case) to the Hugging Face Model Hub.
# The argument "edumunozsala/adapter-phi-3-mini-py_code" is the name of the repository on the Hugging Face Model Hub where the adapter will be saved.
# 'trainer.push_to_hub' は、学習済みモデル（この場合はアダプター）を Hugging Face Model Hub にアップロードするためのメソッド
# 引数 "edumunozsala/adapter-phi-3-mini-py_code" は、アダプターが保存 Hugging Face Model Hub 上のリポジトリ名
trainer.push_to_hub("HuggingFaceUser/adapter-name")

### モデルとアダプターの保存前にメモリを開放
Merge the model and the adapter and save it

Combine the model and the adapter, then save it.  
モデルとアダプターを結合してから保存してください。  
It's necessary to clear the memory when operating on a T4 instance.  
T4インスタンス上で操作する際には、メモリを解放することが必要。

In [ ]:
# This code block is used to free up GPU memory.
# このコードブロックはGPUメモリを解放するために使用

# 'del model' and 'del trainer' are used to delete the 'model' and 'trainer' objects. 
# This removes the references to these objects, allowing Python's garbage collector to free up the memory they were using.
# 'del model' および 'del trainer' は、'model' および 'trainer' オブジェクトを削除するために使用
# これにより、これらのオブジェクトへの参照が削除され、Pythonのガベージコレクタがそれらの使用していたメモリを解放できる
del model
del trainer

# 'import gc' is used to import Python's garbage collector module.
# 'import gc' は、Pythonのガベージコレクタモジュールをインポートする
import gc

# 'gc.collect()' is a method that triggers a full garbage collection, which can help to free up memory.
# It's called twice here to ensure that all unreachable objects are collected.
# 'gc.collect()' は、完全なガベージコレクションを強制的に実行するメソッド。
# これによりメモリの解放が促進される。2回呼び出し到達不能なオブジェクトも確実に回収
gc.collect()
gc.collect()

In [ ]:
# 'torch.cuda.empty_cache()' is a PyTorch method that releases all unoccupied cached memory currently held by 
# the caching allocator so that those can be used in other GPU application and visible in nvidia-smi.
# 'torch.cuda.empty_cache()' は PyTorch のメソッドで、現在保持されている未使用のキャッシュメモリを解放する。
# これにより他の GPU アプリケーションでそのメモリを使用できるようになり、nvidia-smi 上でも確認できるようになる。
torch.cuda.empty_cache()

In [ ]:
# torch.cuda.empty_cache()の実行後にさらに実行すると意味がある
gc.collect()

### モデルを結合し完全なモデルとして保存
以前に学習・保存されたモデルを読み込み、それらを結合して、完全なモデルとして保存  
Load the previously trained and stored model, combine it, and then save the complete model.

In [ ]:
# This code block is used to load the trained model, merge it, and save the merged model.
# このコードブロックは、学習済みモデルを読み込み、マージして、マージ後のモデルを保存

# 'AutoPeftModelForCausalLM' is a class from the 'peft' library that provides a causal language model with PEFT (Performance Efficient Fine-Tuning) support.
# 'AutoPeftModelForCausalLM' は、PEFT（効率的なファインチューニング）に対応した因果言語モデルを提供する 'peft' ライブラリのクラス
from peft import AutoPeftModelForCausalLM

# 'AutoPeftModelForCausalLM.from_pretrained' is a method that loads a pre-trained model (adapter model) and its base model.
#  The adapter model is loaded from 'args.output_dir', which is the directory where the trained model was saved.
# 'low_cpu_mem_usage' is set to True, which means that the model will use less CPU memory.
# 'return_dict' is set to True, which means that the model will return a 'ModelOutput' (a named tuple) instead of a plain tuple.
# 'torch_dtype' is set to 'torch.bfloat16', which means that the model will use bfloat16 precision for its computations.
# 'trust_remote_code' is set to True, which means that the model will trust and execute remote code.
# 'device_map' is the device map that will be used by the model.

# 'AutoPeftModelForCausalLM.from_pretrained' は、事前学習済みモデル（アダプターモデル）とそのベースモデルを読み込むメソッド
# アダプターモデルは 'args.output_dir' （学習済みモデルが保存されているディレクトリ）から読み込まれる。
# ・'low_cpu_mem_usage' を True に設定すると、モデルはCPUメモリの使用量を抑える。
# ・'return_dict' を True に設定すると、モデルは普通のタプルではなく 'ModelOutput'（名前付きタプル）を返す。
# ・'torch_dtype' を 'torch.bfloat16' に設定すると、モデルはbfloat16精度で計算を行う。
# ・'trust_remote_code' を True に設定すると、リモートコードの実行を信頼して許可する。
# ・'device_map' はモデルが使用するデバイスのマッピングを指定する。
new_model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16, #torch.float16,
    trust_remote_code=True,
    device_map=device_map,
)

# 'new_model.merge_and_unload' is a method that merges the model and unloads it from memory.
# The merged model is stored in 'merged_model'.
# 'new_model.merge_and_unload' は、モデルをマージし、メモリからアンロードするメソッド
# マージされたモデルは 'merged_model' に格納される。
merged_model = new_model.merge_and_unload()

# 'merged_model.save_pretrained' is a method that saves the merged model.
# The model is saved in the directory "merged_model".
# 'trust_remote_code' is set to True, which means that the model will trust and execute remote code.
# 'safe_serialization' is set to True, which means that the model will use safe serialization.

# 'merged_model.save_pretrained' は、マージされたモデルを保存するメソッドで、"merged_model" ディレクトリに保存。
# ・'trust_remote_code' を True に設定すると、リモートコードの実行を信頼して許可
# ・'safe_serialization' を True に設定すると、安全なシリアライズ方式で保存
merged_model.save_pretrained("merged_model", trust_remote_code=True, safe_serialization=True)

# 'tokenizer.save_pretrained' is a method that saves the tokenizer.
# The tokenizer is saved in the directory "merged_model".
# 'tokenizer.save_pretrained' は、トークナイザーを保存するメソッド
# トークナイザーは "merged_model" ディレクトリに保存される
tokenizer.save_pretrained("merged_model")

In [ ]:
# This code block is used to push the merged model and the tokenizer to the Hugging Face Model Hub.
# このコードブロックは、マージ済みモデルとトークナイザーを Hugging Face Model Hub にプッシュ（アップロード）する

# 'merged_model.push_to_hub' is a method that pushes the merged model to the Hugging Face Model Hub.
# 'hf_model_repo' is the name of the repository on the Hugging Face Model Hub where the model will be saved.
# 'merged_model.push_to_hub' は、マージ済みモデルを Hugging Face Model Hub にプッシュするメソッド
# 'hf_model_repo' は、モデルが保存される Hugging Face Model Hub 上のリポジトリ名
merged_model.push_to_hub(hf_model_repo)

# 'tokenizer.push_to_hub' is a method that pushes the tokenizer to the Hugging Face Model Hub.
# 'hf_model_repo' is the name of the repository on the Hugging Face Model Hub where the tokenizer will be saved.
# 'tokenizer.push_to_hub' は、トークナイザーを Hugging Face Model Hub にプッシュするメソッド
# 'hf_model_repo' は、トークナイザーが保存される Hugging Face Model Hub 上のリポジトリ名
tokenizer.push_to_hub(hf_model_repo)

## モデルの推論と評価の準備
For model inference and evaluation, we will download the model we created from the Hugging Face Hub and test it to ensure its functionality.  
モデルの推論と評価を行うために、作成したモデルをHugging Face Hubからダウンロードし、正常に機能するかをテストする。

### リポジトリを設定する。

In [ ]:
# 'hf_model_repo' is a variable that holds the name of the repository on the Hugging Face Model Hub.
# This is where the trained and merged model, as well as the tokenizer, have been saved.
# 'hf_model_repo' は、Hugging Face Model Hub 上のリポジトリ名を保持する変数
# このリポジトリには、学習およびマージされたモデルやトークナイザーが保存されている。
hf_model_repo

In [ ]:
# 'hf_model_repo' is a variable that holds the name of the repository on the Hugging Face Model Hub.
# This is where the trained and merged model, as well as the tokenizer, have been saved.
# If 'hf_model_repo' is not defined, it is set to 'username/modelname'.
# This is the default repository where the model and tokenizer will be saved if no other repository is specified.

# 'hf_model_repo' はHugging Face Model Hub上のリポジトリ名を保持する変数で、ここに、訓練済みおよび統合されたモデル、およびトークナイザーが保存されている。
# 'hf_model_repo' が定義されていない場合、'username/modelname' に設定される。これは、デフォルトのリポジトリ。
# hf_model_repo = 'username/modelname' if not hf_model_repo else hf_model_repo # 初期化の段階でリテラルのようになるように変更したので不要になった。

### Hugging Face Hub からモデルとトークナイザーを取得
Retrieve the model and tokenizer from the Hugging Face Hub.

In [ ]:
# This code block is used to load the model and tokenizer from the Hugging Face Model Hub.
# このコードブロックは、Hugging Face Model Hub からモデルとトークナイザーを読み込むために使用

# 'torch' is a library that provides a wide range of functionalities for tensor computations with strong GPU acceleration support.
# 'AutoTokenizer' and 'AutoModelForCausalLM' are classes from the 'transformers' library that provide a tokenizer and a causal language model, respectively.
# 'set_seed' is a function from the 'transformers' library that sets the seed for generating random numbers, which can be used for reproducibility.

# 'torch' は、GPUによる強力な加速サポートを備えたテンソル計算のためのさまざまな機能を提供するライブラリ
# 'AutoTokenizer' と 'AutoModelForCausalLM' は、'transformers' ライブラリからのクラスで、それぞれトークナイザーと因果関係言語モデルを提供
# 'set_seed' は、乱数を生成するためのシードを設定する 'transformers' ライブラリの関数で、再現性を確保するために使用
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed

# 'set_seed(1234)' sets the seed for generating random numbers to 1234.
# 'set_seed(1234)' は、乱数生成のためのシードを 1234 に設定
set_seed(1234)  # For reproducibility # 再現性のため

# 'AutoTokenizer.from_pretrained' is a method that loads a pre-trained tokenizer.
# The tokenizer is loaded from 'hf_model_repo', which is the name of the repository on the Hugging Face Model Hub where the tokenizer was saved.
# 'trust_remote_code' is set to True, which means that the tokenizer will trust and execute remote code.

# 'AutoTokenizer.from_pretrained' は、事前学習済みトークナイザーを読み込むためのメソッド
# トークナイザーは Hugging Face Model Hubのリポジトリ名：'hf_model_repo' から読み込まれる。
# 'trust_remote_code' = True は、トークナイザーがリモートコードを信頼して実行
tokenizer = AutoTokenizer.from_pretrained(hf_model_repo,trust_remote_code=True)

# 'AutoModelForCausalLM.from_pretrained' is a method that loads a pre-trained causal language model.
# The model is loaded from 'hf_model_repo', which is the name of the repository on the Hugging Face Model Hub where the model was saved.
# ・'trust_remote_code' is set to True, which means that the model will trust and execute remote code.
# ・'torch_dtype' is set to "auto", which means that the model will automatically choose the data type for its computations.
# ・'device_map' is set to "cuda", which means that the model will use the CUDA device for its computations.

# 'AutoModelForCausalLM.from_pretrained' は、事前学習済み因果関係言語モデルを読み込むためのメソッドです。
# モデルは Hugging Face Model Hubのリポジトリ名：'hf_model_repo' から読み込まれる。
# ・'trust_remote_code' = True は、モデルがリモートコードを信頼して実行
# ・'torch_dtype' = "auto" は、モデルが自動的に計算のためのデータ型を選択
# ・'device_map' = "cuda" は、モデルがCUDAデバイスを使用して計算を行う
model = AutoModelForCausalLM.from_pretrained(hf_model_repo, trust_remote_code=True, torch_dtype="auto", device_map="cuda")

### データセットを前と同じ方法で整理
We arrange the dataset in the same manner as before.

In [ ]:
# This code block is used to prepare the dataset for model training.
# このコードブロックはモデル学習用のデータセットを準備する

# 'dataset.map(create_message_column)' applies the 'create_message_column' function to each element in the 'dataset'.
# This function is used to create a new column in the dataset.

# 'dataset.map(create_message_column)' は 'dataset' の各要素に 'create_message_column' 関数を適用
# この関数はデータセットに新しい列を作成する
dataset_chatml = dataset.map(create_message_column)

# 'dataset_chatml.map(format_dataset_chatml)' applies the 'format_dataset_chatml' function to each element in 'dataset_chatml'.
# This function is used to format the dataset in a way that is suitable for chat ML.
# 'dataset_chatml.map(format_dataset_chatml)' は 'dataset_chatml' の各要素に 'format_dataset_chatml' 関数を適用
# この関数はチャット向けの機械学習に適した形式にデータセットを整形する
dataset_chatml = dataset_chatml.map(format_dataset_chatml)

# 'dataset_chatml.train_test_split(test_size=0.05, seed=1234)' splits 'dataset_chatml' into a training set and a test set.
# 'test_size=0.05' means that 5% of the data will be used for the test set.
# 'seed=1234' is used for reproducibility.

# 'dataset_chatml.train_test_split(test_size=0.05, seed=1234)' は 'dataset_chatml' を訓練用データとテスト用データに分割
# 'test_size=0.05' はデータの5%をテストセットとして使用
# 'seed=1234' は再現性を確保するために使用
dataset_chatml = dataset_chatml.train_test_split(test_size=0.05, seed=1234)

# 'dataset_chatml' is printed to the console to inspect its contents.
# 'dataset_chatml' の内容を確認するためにコンソールに出力します。
dataset_chatml

In [ ]:
# 'dataset_chatml['test'][0]' is used to access the first element of the test set in the 'dataset_chatml' dataset.
# This can be used to inspect the first test sample to understand its structure and contents.
# 'dataset_chatml['test'][0]' は、'dataset_chatml' データセットのテストセット内の最初の要素にアクセス、最初のテストサンプルの構造や内容を確認
dataset_chatml['test'][0]

### 推論を実行するためのテキスト生成パイプラインを作成
Create a text generation pipeline to run the inference

※ 純粋な推論の実行なので、pipelineはココが初出

In [ ]:
# 'pipeline' is a function from the 'transformers' library that creates a pipeline for text generation.
# 'text-generation' is the task that the pipeline will perform.
# 'model' is the pre-trained model that the pipeline will use.
# 'tokenizer' is the tokenizer that the pipeline will use to tokenize the input text.
# The created pipeline is stored in the 'pipe' variable.

# 'pipeline' は 'transformers' ライブラリの関数で、テキスト生成のためのパイプラインを作成
# ・'text-generation' はパイプラインが実行するタスク
# ・'model' はパイプラインで使用する事前学習済みモデル
# ・'tokenizer' は入力テキストをトークン化するためにパイプラインで使用するトークナイザー
# ・作成されたパイプラインは 'pipe' 変数に保存される
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
# This code block is used to test the chat template.
# このコードブロックは、チャットテンプレートのテストに使用される。

# 'pipe.tokenizer.apply_chat_template' is a method that applies the chat template to a list of messages.
# The list of messages is [{"role": "user", "content": dataset_chatml['test'][0]['messages'][0]['content']}], which is the first message in the test set of 'dataset_chatml'.
# 'tokenize' is set to False, which means that the method will not tokenize the messages.
# 'add_generation_prompt' is set to True, which means that the method will add a generation prompt to the messages.

# 'pipe.tokenizer.apply_chat_template' は、メッセージのリストにチャットテンプレートを適用するメソッド
# メッセージのリストは [{"role": "user", "content": dataset_chatml['test'][0]['messages'][0]['content']}] であり、
# ・これは 'dataset_chatml' のテストセットに含まれる最初のメッセージ
# ・'tokenize' を False に設定すると、メソッドはメッセージをトークン化しない。
# ・'add_generation_prompt' を True に設定すると、メッセージに生成用プロンプトが追加される。
pipe.tokenizer.apply_chat_template([{"role": "user", "content": dataset_chatml['test'][0]['messages'][0]['content']}], tokenize=False, add_generation_prompt=True)

### 入力を整理し、個々のサンプルに対して推論を実行する関数を開発
Develop a function that organizes the input and performs inference on an individual sample.

In [ ]:
# This code block defines a function 'test_inference' that performs inference on a given prompt.
# このコードブロックでは、与えられたプロンプトに対して推論を行う関数 'test_inference' を定義

# 'prompt' is the input to the function. It is the text that the model will generate a response to.
# 'prompt' は関数への入力で、モデルが応答を生成するためのテキスト

# 'pipe.tokenizer.apply_chat_template' is a method that applies the chat template to the prompt.
# The prompt is wrapped in a list and formatted as a dictionary with "role" set to "user" and "content" set to the prompt.
# 'tokenize' is set to False, which means that the method will not tokenize the prompt.
# 'add_generation_prompt' is set to True, which means that the method will add a generation prompt to the prompt.
# The formatted prompt is stored back in the 'prompt' variable.

# 'pipe.tokenizer.apply_chat_template' は、チャットテンプレートをプロンプトに適用するメソッド
# ・'prompt' はリストにラップされ「role」を "user" に、「content」をプロンプトの内容に設定した辞書
# ・'tokenize' は False に設定されており、プロンプトはトークナイズされない。
# ・'add_generation_prompt' は True に設定されており、生成プロンプトが追加される。
# ・フォーマットされたプロンプトは再び 'prompt' 変数に格納される。

# 'pipe' is the text generation pipeline that was created earlier.
# It is called with the formatted prompt and several parameters that control the text generation process.
# 'max_new_tokens=256' limits the maximum number of new tokens that can be generated.
# 'do_sample=True' enables sampling, which means that the model will generate diverse responses.
# 'num_beams=1' sets the number of beams for beam search to 1, which means that the model will generate one response.
# 'temperature=0.3' controls the randomness of the responses. Lower values make the responses more deterministic.
# 'top_k=50' limits the number of highest probability vocabulary tokens to consider for each step.
# 'top_p=0.95' enables nucleus sampling and sets the cumulative probability of parameter tokens to 0.95.
# 'max_time=180' limits the maximum time for the generation process to 180 seconds.
# The generated responses are stored in the 'outputs' variable.

# 'pipe' は以前に作成されたテキスト生成パイプライン
# フォーマットされたプロンプトと、テキスト生成プロセスを制御するいくつかのパラメタとともに呼び出される。
# ・'max_new_tokens=256' は、生成される新しいトークンの最大数を 256 に制限
# ・'do_sample=True' はサンプリングを有効にし、多様な応答を生成
# ・'num_beams=1' はビームサーチのビーム数を 1 に設定し、1 つの応答のみを生成
# ・'temperature=0.3' は応答のランダム性を制御。値が小さいほど応答は決定論的。
# ・'top_k=50' は各ステップで考慮する語彙の上位確率トークン数を 50 に制限
# ・'top_p=0.95' は nucleus sampling を有効にし、累積確率が 0.95 になるようにトークンを選ぶ。
# ・'max_time=180' は生成処理の最大時間を 180 秒に制限
# ・生成された応答は 'outputs' 変数に格納される。

# The function returns the first generated response.
# The response is stripped of the prompt and any leading or trailing whitespace.
# この関数はプロンプト部分と前後の空白を取り除いてから最初の生成応答を返す。
def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, num_beams=1, temperature=0.3, top_k=50, top_p=0.95,
                   max_time= 180) #, eos_token_id=eos_token)
    return outputs[0]['generated_text'][len(prompt):].strip()

### 準備モデルの推論を試行

In [ ]:
# This code block calls the 'test_inference' function with the first message in the test set of 'dataset_chatml' as the prompt.
# 'test_inference' performs inference on the prompt and returns a generated response.
# The response is printed to the console.

# このコードブロックは、'dataset_chatml' のテストセットの最初のメッセージをプロンプトとして 'test_inference' 関数を呼び出す
# 'test_inference' はそのプロンプトに対して推論を行い、生成された応答を返す。応答はコンソールに出力される。
test_inference(dataset_chatml['test'][0]['messages'][0]['content'])

### モデルの評価メトリックの読み込み

In [ ]:
# 'load_metric' is a function from the 'datasets' library that loads a metric for evaluating the model.
# Metrics are used to measure the performance of the model on certain tasks.
# 'load_metric' は 'datasets' ライブラリからの関数で、モデルの評価に使用するメトリックを読み込む
# メトリックは、特定のタスクにおけるモデルのパフォーマンスを測定するために使用される。
from datasets import load_metric

# 'load_metric("rouge", trust_remote_code=True)' loads the ROUGE metric from the 'datasets' library.
# ROUGE is a set of metrics used to evaluate automatic summarization and machine translation.
# We'll employ the ROUGE metric to assess performance. While it may not be the optimal metric, it's straightforward and convenient to utilize.
# 'trust_remote_code' is set to True, which means that the metric will trust and execute remote code.
# The loaded metric is stored in the 'rouge_metric' variable.

# 'load_metric("rouge", trust_remote_code=True)' は ROUGE 指標を読み込む
# パフォーマンス評価にはROUGE指標を使用、自動要約や機械翻訳の評価に使用される一連の指標で、最適ではないかもしれないが、シンプルで使いやすい指標
# 'trust_remote_code' を True に設定することで、リモートコードを信頼
# 読み込まれた指標は 'rouge_metric' 変数に格納される。
rouge_metric = load_metric("rouge", trust_remote_code=True)

### 推論を実行し、インスタンスを評価するための関数を開発
Develop a function for performing inference and assessing an instance.

In [ ]:
# This code block defines a function 'calculate_rogue' that calculates the ROUGE score for a given row in the dataset.
# このコードブロックは、データセットの各行に対して ROUGE スコアを計算する関数 'calculate_rogue' を定義

# 'row' is the input to the function. It is a row in the dataset that contains a message and its corresponding output.
# 'row' は関数への入力でメッセージとそれに対応する出力を含むデータセットの1行

# 'test_inference(row['messages'][0]['content'])' calls the 'test_inference' function with the first message in the row as the prompt.
# 'test_inference' performs inference on the prompt and returns a generated response.
# The response is stored in the 'response' variable.

# 'test_inference(row['messages'][0]['content'])' は、その行の最初のメッセージをプロンプトとして 'test_inference' 関数を呼び出す。
# 'test_inference' はプロンプトに対して推論を行い、生成された応答を返し 'response' という変数に保存。

# 'rouge_metric.compute' is a method that calculates the ROUGE score for the generated response and the corresponding output in the row.
# 'predictions' is set to the generated response and 'references' is set to the output in the row.
# 'use_stemmer' is set to True, which means that the method will use a stemmer to reduce words to their root form.
# The calculated ROUGE score is stored in the 'result' variable.

# 'rouge_metric.compute' は、生成された応答と行に含まれる正解出力との ROUGE スコアを計算するメソッド
# 'predictions' には生成された応答を、'references' には正解出力を指定
# 'use_stemmer' を True に設定することで、単語を語幹に変換して比較
# 計算された ROUGE スコアは 'result' 変数に保存

# The 'result' dictionary is updated to contain the F-measure of each ROUGE score multiplied by 100.
# The F-measure is a measure of a test's accuracy that considers both the precision and the recall of the test.

# 'result' 辞書は、各 ROUGE スコアの F値（F-measure）に100を掛けた値（%）を持つように更新
# F値は、精度（precision）と再現率（recall）の両方を考慮したテストの精度指標です。

# The 'response' is added to the 'result' dictionary.
# 'response' を 'result' 辞書に追加

# The function returns the 'result' dictionary.
def calculate_rogue(row):
    response = test_inference(row['messages'][0]['content'])
    result = rouge_metric.compute(predictions=[response], references=[row['output']], use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    result['response']=response
    return result

# 関数は 'result' 辞書を返します。
def calculate_rogue(row):
    response = test_inference(row['messages'][0]['content'])
    result = rouge_metric.compute(predictions=[response], references=[row['output']], use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    result['response'] = response
    return result

## モデルの推論と評価の実行

Now, we have the ability to execute inference on a collection of samples. For simplicity, the process isn't optimized at this stage. In the future, we plan to perform inference in batches to enhance performance. However, for the time being,
現在、複数のサンプルに対して推論を実行できるようになりました。簡略化のため、現時点ではプロセスは最適化されていません。将来的には、パフォーマンスを向上させるために推論をバッチ処理で実行する予定です。ただし、当面は。

In [ ]:
# '%%time' is a magic command in Jupyter notebooks that measures the execution time of the cell.
# '%%time'はJupyterノートブックのマジックコマンドで、セルの実行時間を測定
%%time

# 'dataset_chatml['test'].select(range(0,500))' selects the first 500 elements from the test set in the 'dataset_chatml' dataset.
# 'dataset_chatml['test'].select(range(0,500))'は、'dataset_chatml'データセットのテストセットから最初の500要素を選択

# '.map(calculate_rogue, batched=False)' applies the 'calculate_rogue' function to each element in the selected subset.
# 'calculate_rogue' calculates the ROUGE score for each element.
# 'batched' is set to False, which means that the function will be applied to each element individually, not in batches.

# '.map(calculate_rogue, batched=False)'は、選択したサブセットの各要素に'calculate_rogue'関数を適用
# 'calculate_rogue'は、各要素に対してROUGEスコアを計算
# 'batched'がFalseに設定されているため、関数はバッチ処理ではなく、各要素に個別に適用

# The results are stored in the 'metricas' variable.
# 結果は'metricas'変数に格納されます。
metricas = dataset_chatml['test'].select(range(0,500)).map(calculate_rogue, batched=False)

In [ ]:
# 'numpy' is a library in Python that provides support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.
# 'import numpy as np' imports the 'numpy' library and gives it the alias 'np'. This allows us to use 'np' instead of 'numpy' when calling its functions.
# 'numpy'はPythonのライブラリで、大規模な多次元配列や行列のサポートを提供し、これらの配列に対して操作を行うための多くの高レベルな数学関数を提供します。
# 'import numpy as np'は'numpy'ライブラリをインポートし、それにエイリアス'numpy'の代わりに使用できる'np'を付けます。これにより、関数を呼び出す際に'numpy'の代わりに'np'を使用できます。
import numpy as np

# This code block prints the mean of the ROUGE-1, ROUGE-2, ROUGE-L, and ROUGE-Lsum scores in the 'metricas' dictionary.
# このコードブロックは、'metricas' 辞書に含まれる ROUGE-1、ROUGE-2、ROUGE-L、および ROUGE-Lsum スコアの平均を出力

# 'np.mean(metricas['rouge1'])' calculates the mean of the ROUGE-1 scores. ROUGE-1 スコアの平均を計算
# 'np.mean(metricas['rouge2'])' calculates the mean of the ROUGE-2 scores. ROUGE-2 スコアの平均を計算
# 'np.mean(metricas['rougeL'])' calculates the mean of the ROUGE-L scores. ROUGE-L スコアの平均を計算
# 'np.mean(metricas['rougeLsum'])' calculates the mean of the ROUGE-Lsum scores. ROUGE-Lsum スコアの平均を計算

# 'print' is used to print the calculated means to the console.
# 'print' を使用して、計算された平均値をコンソールに表示します。

print("Rouge 1 Mean: ",np.mean(metricas['rouge1']))
print("Rouge 2 Mean: ",np.mean(metricas['rouge2']))
print("Rouge L Mean: ",np.mean(metricas['rougeL']))
print("Rouge Lsum Mean: ",np.mean(metricas['rougeLsum']))     